# Training a Neural Network for Computer Model Emulation

### Import necessary libraries 

In [1]:
%matplotlib inline
import sklearn
import numpy as np
import matplotlib.pyplot as plt
from warnings import filterwarnings
filterwarnings('ignore')

import seaborn as sns
sns.set()
from pandas.plotting import scatter_matrix
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
from calculations_load import *
from configurations import *

Using idf = 0 : Grad
SystemsInfo = 
{'Pb-Pb-2760': {'proj': 'Pb', 'targ': 'Pb', 'sqrts': 2760, 'main_design_file': 'production_designs/500pts/design_pts_Pb_Pb_2760_production/design_points_main_PbPb-2760.dat', 'main_range_file': 'production_designs/500pts/design_pts_Pb_Pb_2760_production/design_ranges_main_PbPb-2760.dat', 'validation_design_file': 'production_designs/500pts/design_pts_Pb_Pb_2760_production/design_points_validation_PbPb-2760.dat', 'validation_range_file': 'production_designs/500pts/design_pts_Pb_Pb_2760_production//design_ranges_validation_PbPb-2760.dat', 'main_events_dir': 'model_calculations/production_500pts_Pb_Pb_2760/Events/main', 'validation_events_dir': 'model_calculations/production_500pts_Pb_Pb_2760/Events/validation', 'main_obs_file': 'model_calculations/production_500pts_Pb_Pb_2760/Obs/main.dat', 'validation_obs_file': 'model_calculations/production_500pts_Pb_Pb_2760/Obs/validation.dat', 'n_design': 500, 'n_validation': 100, 'design_remove_idx': [289, 483, 32

In [4]:
# Get all the observables list

nobs        =  0
observables =  []
obs_name    =  []

for obs, cent_list in obs_cent_list['Pb-Pb-2760'].items():
    if obs not in active_obs_list['Pb-Pb-2760']:
        continue
    observables.append(obs)
    n = np.array(cent_list).shape[0]
    for i in cent_list:
        obs_name.append(f'{obs}_{i}')
    #self._slices[obs] = slice(self.nobs, self.nobs + n)
    nobs += n

In [5]:
#prepare the directories for model design and outputs
#run this cell only once, or else it will hang!
#! sh prepare.sh

### Loading Computer Model inputs and outputs

In [6]:
system_str = 'Pb-Pb-2760'
design_file = 'production_designs/500pts/design_pts_Pb_Pb_2760_production/design_points_main_PbPb-2760.dat'
design = pd.read_csv(design_file)
design = design.drop("idx", axis=1)

#delete bad design points
drop_indices = list(delete_design_pts_set)
design = design.drop(drop_indices)

#choose features (inputs)
#feature_cols = ['norm', 'trento_p'] #specific choices
feature_cols = design.keys().values #all of them 
n_features = len(feature_cols)

X = design[feature_cols]

n_design = SystemsInfo["Pb-Pb-2760"]["n_design"]
npt = n_design - len(delete_design_pts_set)
obs = 'dNch_deta' #choose the observable we want to emulate

Y = np.array([])

for pt in range(npt):
    for obs in active_obs_list['Pb-Pb-2760']:
        Y = np.append( Y, trimmed_model_data[system_str][pt, idf][obs]['mean'][:], axis=0)
Y = Y.reshape(X.shape[0], -1)

In [7]:
print( "X.shape : "+ str(X.shape) )
print( "Y.shape : "+ str(Y.shape) )

X.shape : (485, 17)
Y.shape : (485, 110)


## Splitting the inputs and outputs into a training and testing set

Then, scaling all of the inputs and outputs to (0, 1)

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2)

#X_scaler = StandardScaler().fit(X_train)
#Y_scaler = StandardScaler().fit(Y_train)
X_scaler = MinMaxScaler(feature_range=(-1, 1)).fit(X_train)
Y_scaler = MinMaxScaler(feature_range=(-1, 1)).fit(Y_train)

X_train_sc = X_scaler.transform(X_train)
X_test_sc = X_scaler.transform(X_test)

Y_train_sc = Y_scaler.transform(Y_train)
Y_test_sc = Y_scaler.transform(Y_test)

### The cell below constructs the neural network

In [29]:
# n_hidden = 0 # number of hidden layers

# model = keras.Sequential()
# model.add(layers.Dense(50, input_dim=X.shape[1], activation='tanh'))
# for n in range(n_hidden):
#     model.add(layers.Dense(20, activation='tanh'))
# model.add(layers.Dropout(0.5))
# model.add(layers.Dense(Y.shape[1], activation='tanh'))
# model.summary()

TypeError: add() got an unexpected keyword argument 'training'

In [151]:
from keras.models import Model
from keras.layers import Flatten, Input, Dense, Dropout ,Conv1D
def model_fn(ly1_units=20,activation_1='sigmoid',activation_2='tanh',ly2_units=20,activation_3='tanh',\
             dropout_rate1 = 0.1,dropout_rate2 = 0.1,loss_fn="huber_loss", krnl_sz=5,\
            optimizer='adam'):
    inputs = Input(shape=(X.shape[1],1))
    x = Dense(ly1_units, activation=activation_1)(inputs)
   # print(x.shape)
    x=  Conv1D(filters=1,kernel_size=krnl_sz)(x)
    x= Flatten()(x)
    x = Dropout(dropout_rate1)(x, training=True)
    x = Dense(ly2_units, activation=activation_2)(x)
    x = Dropout(dropout_rate2)(x, training=True)
    x = Dense(Y.shape[1], activation=activation_3)(x)
    outputs = x
    model = Model(inputs, outputs)
#model.compile(loss="mean_squared_error", optimizer='adam')
    model.compile(loss=loss_fn, optimizer=optimizer)
    model.summary()
    return model

In [152]:
# compile the keras model
#model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model=model_fn()

Model: "model_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_49 (InputLayer)        (None, 17, 1)             0         
_________________________________________________________________
dense_68 (Dense)             (None, 17, 20)            40        
_________________________________________________________________
conv1d_33 (Conv1D)           (None, 13, 1)             101       
_________________________________________________________________
flatten_17 (Flatten)         (None, 13)                0         
_________________________________________________________________
dropout_22 (Dropout)         (None, 13)                0         
_________________________________________________________________
dense_69 (Dense)             (None, 20)                280       
_________________________________________________________________
dropout_23 (Dropout)         (None, 20)                0  

In [160]:
test=np.expand_dims(X_train_sc,axis=2)

In [156]:
from sklearn.model_selection import GridSearchCV
ly_units=[50,100,200,500]
activation_1=['sigmoid','tanh']
activation_2=['linear','tanh']
dropout_rate = [0.2, 0.3, 0.5]
krnl_sz=[5,10,20,40]
loss_fn=["mse","huber_loss"]
optimizer=['adam']
batch_size=[10, 20, 50]
estimator=tf.keras.wrappers.scikit_learn.KerasRegressor(build_fn=model_fn)
param_grid = dict(ly1_units=ly_units,ly2_units=ly_units,activation_1=activation_1, activation_2=activation_1,\
                activation_3=activation_2,dropout_rate1=dropout_rate,dropout_rate2=dropout_rate,\
                  loss_fn=loss_fn,optimizer=optimizer,batch_size=batch_size,krnl_sz=krnl_sz)
grid = GridSearchCV(estimator=estimator, param_grid=param_grid, n_jobs=-1, cv=2, scoring='r2',verbose=20)
grid_result = grid.fit(test, Y_train_sc,epochs=300,verbose=0)

Fitting 2 folds for each of 27648 candidates, totalling 55296 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   23.4s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   23.9s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:   24.0s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   24.4s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   25.6s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:   28.1s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:   28.9s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   29.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   44.6s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   45.0s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:   47.8s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   49.0s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   50.3s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   50.5s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:   

KeyboardInterrupt: 

In [ ]:
best_params_=grid_result.best_params_

In [ ]:
#
#{'activation_1': 'tanh',
#  'activation_2': 'tanh',
#  'batch_size': 10,
#  'dropout_rate': 0.2,
#  'loss_fn': 'mse',
#  'ly1_units': 200,
#  'optimizer': 'adam'}

In [ ]:
best_params_

In [ ]:
best_params_.pop('batch_size')

In [ ]:
model_b=model_fn(**best_params_)

In [ ]:
model_b.fit(X_train_sc, Y_train_sc, epochs=500, batch_size=10,verbose=0)

In [ ]:
import numpy as np
predictions = []
sample_size = 1000
for t in range(sample_size):
    predictions.append(model.predict(X_test_sc))
   # pred_array = np.array(predictions)
   # print(pred_array.shape)
#prediction_df = pd.DataFrame()
#prediction_std= pd.DataFrame()
pred_array = np.array(predictions)
prediction_df= np.mean(pred_array,axis=0)
prediction_sd= np.std(pred_array,axis=0)
#prediction_df['std'] = pred_array.std(axis=0)

In [ ]:
truth = np.linspace(-1, 1, 100)
obs_indices = [60, 61, 68, 69, 15, 85]

fig, axes = plt.subplots(2, 3, figsize=(12, 7))
fig.suptitle('Prediction vs truth with 2 Sigma uncertinity', fontsize=16)
fig.text(0.5, 0.04, 'Truth', ha='center')
fig.text(0.04, 0.5, 'Prediction', va='center', rotation='vertical')
for i, obs_ind in enumerate(obs_indices):
    ax = axes.flatten()[i]
    ax.set_title(obs_name[obs_ind])
    ax.plot(truth, truth, c='r', lw=3, zorder=-1)
    ax.errorbar(Y_test_sc[:, obs_ind], prediction_df[:, obs_ind],yerr=2*prediction_sd[:,obs_ind],alpha=0.5,fmt='x')

In [ ]:
from sklearn.metrics import r2_score
coefficient_of_dermination = r2_score(Y_test_sc,prediction_df)
coefficient_of_dermination_tf = r2_score(Y_scaler.inverse_transform(Y_test_sc),Y_scaler.inverse_transform(prediction_df))

In [ ]:
print(f'coefficient_of_dermination transformed_observable space {coefficient_of_dermination_tf}, normalized space {coefficient_of_dermination}')

In [ ]:
import numpy as np
def root_mean_squre_percentage_error(y_true, y_pred,feature_loc): 
    y_true, y_pred = Y_scaler.inverse_transform(y_true)[:,feature_loc], Y_scaler.inverse_transform(y_pred)[:,feature_loc]
    return np.sqrt(np.mean(np.square((y_true - y_pred)/ y_true))) * 100

In [ ]:
#from sklearn.metrics import mean_squared_error
def plot_rmse_bp(observable='dN_dy_pion'):
    # Get all the observables list
    prediction_df_bp=pd.DataFrame()
    nobs        =  0
    observables =  []
    obs_name    =  []
    prediction_rmse=pd.DataFrame()
    for obs, cent_list in obs_cent_list['Pb-Pb-2760'].items():
        nobs += nobs
        if obs !=observable:
            continue
        for k,i in enumerate(cent_list):
            obs_name.append(f'{obs}_{i}')
            new=prediction_df[:,nobs+k]-Y_test_sc[:,nobs+k]
        #self._slices[obs] = slice(self.nobs, self.nobs + n)
          #  print(np.mean(-Y_test_sc[:,i],axis=0))
           # print(prediction_df[:,i]-Y_test_sc[:,i])
            prediction_df_bp[f'{i}']=new
            #print(mean_squared_error(prediction_df[:,nobs+k],Y_test_sc[:,nobs+k]))
            prediction_rmse[f'{i}']=[root_mean_squre_percentage_error(Y_test_sc,prediction_df,nobs+k)]
          #  prediction_df_bp[i]=
          #  print(prediction_df_bp.shape)
    fig, axes = plt.subplots(2, 1, figsize=(20, 10))
    for i in [0,1]:
        ax=axes.flatten()[i]
        if (i==0):
            boxplot = prediction_df_bp.boxplot(rot=90,ax=ax,showfliers=False)
            ax.set_xticklabels([])
            ax.set_ylabel('Normalized Residuals',fontsize=15)
        if (i==1):
            ax.scatter(prediction_rmse.keys(),prediction_rmse.iloc[0])
            ax.set_xticklabels(labels=prediction_rmse.keys(), rotation=90, ha='right')
            ax.set_ylabel('RMS % error',fontsize=15)
            #scatter_plt = prediction_rmse.plot.scatter(rot=90,ax=ax)
    fig.text(0.04, 0.5, observable, va='center', rotation='vertical',fontsize=16)

In [ ]:
%matplotlib inline
for obs in active_obs_list['Pb-Pb-2760']:
    plot_rmse_bp(obs)